In [ ]:
import numpy as np
import gdspy
import matplotlib.pyplot as plt
from cytoolz import dissoc

%matplotlib inline
from functools import partial

In [ ]:
%load_ext autoreload
%autoreload 2
import snakenbake as s
from geometry import Cell, Round

In [ ]:
import pint

u = pint._DEFAULT_REGISTRY

In [ ]:
def plot_cell(cell, exclude=(2,)):
    # FROM: https://github.com/heitzmann/gdspy/issues/42
    if hasattr(cell, "get_polygons"):
        poly_dict = cell.get_polygons(by_spec=True)
    else:
        poly_dict = {
            l_d: [p] for l_d, p in zip(zip(cell.layers, cell.datatypes), cell.polygons)
        }
    plt.figure(figsize=(40, 20))
    for layer_datatype, polys in poly_dict.items():
        if layer_datatype[0] in exclude:
            continue
        for poly in polys:
            plt.fill(*poly.T, lw=0.5, ec="k", fc=(1, 0, 0, 0.5))
    plt.axes().set_aspect("equal", "datalim")


def write_gds(main_cell, filename, unit=1.0e-6, precision=1.0e-9):
    # writer = gdspy.GdsWriter(filename, unit=unit, precision=precision)
    # for cell in [main_cell] + list(main_cell.get_dependencies(True)):
    #    writer.write_cell(cell)
    # writer.close()
    cells = [main_cell] + list(main_cell.get_dependencies(True))
    gdspy.write_gds(filename, cells=cells, unit=unit, precision=precision)

In [ ]:
def fovs_per_chip(
    fov,
    feeding_channel_width=None,
    trench_length=None,
    trench_gap=None,
    num_lanes=None,
    lane_with_trenches_length=None,
    **kwargs,
):
    hfovs = int(np.ceil(lane_with_trenches_length / fov[0]))
    y = 2 * trench_length + trench_gap
    unit_cell_height = y + feeding_channel_width
    trench_sets_per_fov = 2
    while True:
        delta_y = feeding_channel_width + trench_length
        if y + delta_y < fov[1]:
            y += delta_y
            unit_cell_height = y + trench_gap
            trench_sets_per_fov += 1
        else:
            break
        delta_y = trench_gap + trench_length
        if y + delta_y < fov[1]:
            y += delta_y
            unit_cell_height = y + feeding_channel_width
            trench_sets_per_fov += 1
        else:
            break
    vfovs = int(np.ceil(num_lanes * 2 / trench_sets_per_fov))
    return np.array([hfovs, vfovs]), unit_cell_height, y

In [ ]:
def print_fovs(fov_dims, metadata):
    for fov_name, fov_dim in fov_dims.items():
        fovs, unit_cell_height, active_height = fovs_per_chip(fov_dim, **metadata)
        print(f"{fov_name}: {fov_dim[0]} x {fov_dim[1]}")
        # print(metadata)
        print("    FOVs: {} ({} x {})".format(np.product(fovs), fovs[0], fovs[1]))
        print("    unit cell height:", unit_cell_height)
        print(
            "    active height: {} (margin: {})".format(
                active_height, fov_dim[1] - active_height
            )
        )

In [ ]:
camera_dims = {
    "iris": (4.25, np.array([5056, 2960])),
    "bsi": (6.5, np.array([2048, 2048])),
}
fov_dims = {}
for camera, (pixel_size, fov_dim) in camera_dims.items():
    for mag in (20, 40, 60, 100):
        fov_dims[f"{camera} {mag}x"] = fov_dim * pixel_size / mag

# Single-input prototype

In [ ]:
a = gdspy.Round((0, 0), 10, initial_angle=0, final_angle=np.pi)

In [ ]:
(1 / 2 + 1 / 2) + np.array([0, 1 / 2])

In [ ]:
(-1 + np.array([3, 2])) / 2  # * np.pi

In [ ]:
plot_cell(gdspy.Round((0, 0), 10, initial_angle=np.pi, final_angle=np.pi / 2))

In [ ]:
plot_cell(gdspy.Round((0, 0), 10, initial_angle=2 * np.pi, final_angle=3 / 2 * np.pi))

In [ ]:
metadata = {}
metadata2 = {}
# chip_dims = np.array([25e3, 16e3])
# chip_dims = np.array([38e3, 20e3])
chip_dims = np.array([50e3, 50e3])
main_cell = s.chip(
    "Big Snake",
    s.manifold_snake,
    dims=chip_dims,
    split=None,
    manifold_input_style="bend-in",
    num_manifolds=1,
    lanes_per_snake=3,
    manifold_width=200,
    manifold_input_margin=2e3,
    manifold_bend_margin=0.2e3,
    manifold_bend_radius=300,
    manifold_margin=100,
    border_margin=400,
    feeding_channel_width=40,
    trench_spacing=2,
    trench_gap=20,
    trench_width=1.5,
    registration_marks=True,
    # mark_size=2,
    # mark_spacing=2,
    chip_id=2**10 - 1,
    ticks=False,
    port_margin=0.3e3,
    top_margin=0.9e3,
    bottom_margin=0.9e3,
    draw_trenches=False,
    metadata=metadata,
)
# print(metadata)
# plot_cell(main_cell)
# plot_cell(main_cell2)
print(metadata["Big Snake"]["num_trenches"])
# write_gds(main_cell, '200214manifold_test.gds')

In [ ]:
get_resistance(75, metadata["Big Snake"])

In [ ]:
res = manifold_flow_rates(feeding_channel_height=75, **metadata["Big Snake"])

plt.figure(figsize=(12, 8))
for r in res:
    v = r["q"]
    plt.plot(v / v.max())

In [ ]:
np.log2(metadata["Big Snake"]["num_lanes"])

In [ ]:
np.log2(metadata["Big Snake"]["trenches_per_set"] / 20)

In [ ]:
w = 400 * u.um
h = 75 * u.um
D_h = 2 * w * h / (w + h)
Q = 20 * u.uL / u.minute * 30
nu = 0.7 * u.centistokes
Re = Q * D_h / ((w * h) * nu)
L_e = D_h * (0.55 / (0.13 * Re + 1) + 0.065 * Re)

In [ ]:
Re.to_base_units()

In [ ]:
Re

In [ ]:
D_h

In [ ]:
L_e

In [ ]:
6.5 * 2048 / np.array([20, 40, 60, 100])

In [ ]:
4.25 * 2960 / np.array([20, 40, 60, 100])

In [ ]:
6.5 / 40

In [ ]:
4.25 / 20

# Manifold flow

In [ ]:
md = metadata["Big Snake"]
md.keys()

In [ ]:
def _ensure_array(val, size):
    if np.isscalar(val):
        return np.full(size, val)
    else:
        return val


def ladder_flow_rates(R_snake, R_left, R_right, N=None):
    for ary, inc in ((R_snake, 0), (R_left, 1), (R_right, 1)):
        if not np.isscalar(ary):
            if N is None:
                N = len(ary) + inc
            elif N != len(ary) + inc:
                raise ValueError("got conflicting ladder sizes")
    if N is None:
        raise ValueError("ladder size must be specified if resistances are scalars")
    R_snake = _ensure_array(R_snake, N)
    R_left = _ensure_array(R_left, N)
    R_right = _ensure_array(R_right, N)
    A = np.zeros((N, N))
    b = np.zeros(N)
    b[0] = 1
    A[0, :] = 1
    for i in range(1, N):
        A[i, : i - 1] = R_left[i - 1]
        A[i, i - 1] = -(R_right[i - 1] + R_snake[i - 1])
        A[i, i] = R_left[i - 1] + R_snake[i]
        A[i, i + 1 :] = -R_right[i - 1]
    q = np.linalg.solve(A, b)
    return q


def manifold_flow_rates(
    feeding_channel_height=None,
    split=None,
    split_cum=None,
    manifold_split=None,
    manifold_split_cum=None,
    left_port_lanes=None,
    right_port_lanes=None,
    lane_length=None,
    manifold_width=None,
    feeding_channel_width=None,
    lane_ys=None,
    **kwargs,
):
    split_cum = np.concatenate(((0,), np.cumsum(split)))
    manifold_split_cum = np.concatenate(((0,), np.cumsum(manifold_split)))
    results = []
    for manifold in range(len(manifold_split)):
        selected_lanes = slice(
            manifold_split_cum[manifold], manifold_split_cum[manifold + 1]
        )
        lanes_per_snake = split[selected_lanes]
        left_segment_lengths = -np.diff(lane_ys[left_port_lanes[selected_lanes]])
        right_segment_lengths = -np.diff(lane_ys[right_port_lanes[selected_lanes]])
        R_snake = (
            lanes_per_snake
            * lane_length
            * resistance(feeding_channel_height, feeding_channel_width)
        )
        R_left = left_segment_lengths * resistance(
            feeding_channel_height, manifold_width
        )
        R_right = right_segment_lengths * resistance(
            feeding_channel_height, manifold_width
        )
        q = ladder_flow_rates(R_snake, R_left, R_right)
        R = ((1 - np.cumsum(q))[:-1] * R_left).sum() + q[-1] * R_snake[-1]
        results.append(
            {"R_snake": R_snake, "R_left": R_left, "R_right": R_right, "q": q, "R": R}
        )
    return results

In [ ]:
res = manifold_flow_rates(feeding_channel_height=75, **metadata["Big Snake"])

plt.figure(figsize=(12, 8))
for r in res:
    v = r["q"]
    plt.plot(v / v.max())

In [ ]:
# def resistance(h, w, eta=1 * u.centipoise):
def resistance(h, w, eta=1):
    h_ = np.minimum(h, w)
    w_ = np.maximum(h, w)
    h, w = h_, w_
    return 1 / (h**3 * w / (12 * eta) * (1 - 0.630 * h / w))

In [ ]:
h = 50 * u.um

In [ ]:
resistance(h, 100 * u.um) * u.m * (20 * u.uL / u.s)

## S8 chip resistance

In [ ]:
feeding_channel_height = 75  # * u.um
split = metadata["Basilisk S8 FC80 L35 W1.4 TS2.1"]["split"]
lane_length = metadata["Basilisk S8 FC80 L35 W1.4 TS2.1"]["lane_length"]
feeding_channel_width = metadata["Basilisk S8 FC80 L35 W1.4 TS2.1"][
    "feeding_channel_width"
]
R = split * lane_length * resistance(feeding_channel_height, feeding_channel_width)
R

In [ ]:
feeding_channel_height = 75  # * u.um
split = metadata["Basilisk S8 FC40 L35 W1.4 TS1.1"]["split"]
lane_length = metadata["Basilisk S8 FC40 L35 W1.4 TS1.1"]["lane_length"]
feeding_channel_width = metadata["Basilisk S8 FC40 L35 W1.4 TS1.1"][
    "feeding_channel_width"
]
R = split * lane_length * resistance(feeding_channel_height, feeding_channel_width)
R

In [ ]:
s3_metadata = {}
main_cell = s.chip(
    "Big Snake",
    s.snake,
    dims=np.array([23e3, 13e3]),
    split=3,
    gap_lanes=0,
    feeding_channel_width=45,
    trench_spacing=2,
    trench_gap=20,
    trench_width=1.5,
    port_margin=1.5e3,
    top_margin=1.2e3,
    bottom_margin=1.2e3,
    horizontal_margin=4e3,
    draw_trenches=False,
    metadata=s3_metadata,
)

In [ ]:
plot_cell(main_cell)

In [ ]:
s3_metadata

In [ ]:
feeding_channel_height = 100  # * u.um
split = s3_metadata["Big Snake"]["split"]
lane_length = s3_metadata["Big Snake"]["lane_length"]
feeding_channel_width = s3_metadata["Big Snake"]["feeding_channel_width"]
R = split * lane_length * resistance(feeding_channel_height, feeding_channel_width)
R

# Flow calculations

In [ ]:
eta = 1 * u.centipoise

In [ ]:
def Q_over_deltap(eta, L, h, w):
    h_ = np.minimum(h, w)
    w_ = np.maximum(h, w)
    h, w = h_, w_
    return h**3 * w / (12 * eta * L) * (1 - 0.630 * h / w)

In [ ]:
baseline_md = chip_metadata(feeding_channel_width=150, lane_gap=400)
print(baseline_md)

In [ ]:
h = 50 * u.um
w = baseline_md["feeding_channel_width"] * u.um
L = baseline_md["split"][0] * baseline_md["lane_length"] * u.um
baseline_Q_over_deltap = Q_over_deltap(eta, L, h, w)

In [ ]:
h = 50 * u.um
w = np.linspace(50, 150, 100) * u.um
L = 15 * 17000 * u.um
plt.plot(w, baseline_Q_over_deltap / Q_over_deltap(eta, L, h, w))

In [ ]:
np.array((5056, 2960)) * 4.25 / 20

In [ ]:
lane_length = 17615 * u.um
h = 50 * u.um
w = np.linspace(50, 150, 100) * u.um
plt.figure(figsize=(10, 8))
ax = plt.gca()
ax.plot(w, 1 / Q_over_deltap(eta, lane_length * 23, 50, w))
for (lanes, width, height), c in zip(
    ((15, 50, 50), (19, 90, 90), (29, 45, 90), (23, 40, 50), (23, 40, 90)),
    plt.cm.get_cmap("Set1").colors,
):
    v = 1 / Q_over_deltap(eta, lane_length * lanes, height, width)
    ax.axhline(
        v.magnitude,
        c=c,
        ls="--",
        label="{} lane snake ({} um width, {} um height)".format(lanes, width, height),
    )
plt.legend()

In [ ]:
plt.figure(figsize=(10, 8))
hs = np.array([50, 75, 100, 150]) * u.um
w = np.linspace(40, 150, 100) * u.um
chips = [
    chip_metadata(feeding_channel_width=width.magnitude, trench_spacing=2, split=8)
    for width in w
]
L = np.array([np.max(m["split"]) * m["lane_length"] for m in chips]) * u.um
# L = 15 * 17000 * u.um
eta = 1 * u.centipoise
ax = plt.gca()
for h in hs:
    ax.plot(
        w,
        baseline_Q_over_deltap / Q_over_deltap(eta, L, h, w),
        label="feeding channel height: {:~P}".format(h),
    )
ax2 = plt.gca().twinx()
ax2.plot(
    w,
    [m["num_trenches"] for m in chips],
    ls="-.",
    c="purple",
    label="number of trenches (2 um trench spacing)",
)
# ax2.plot(w, [np.max(m['split']) for m in chips], c='r', label='number of lanes')
ax.axhline(1, c="k", ls=":", label="15 lane snake (50 um height)")
ax.legend()
ax.set_ylabel("pressure (relative to 15-lane snake)")
ax.set_xlabel("feeding channel width (um)")
ax2.legend(loc="lower left")

# Designs

In [ ]:
def chip_metadata(**kwargs):
    metadata = {}
    main_cell = s.chip(
        "my_chip", **{"metadata": metadata, **kwargs, "draw_trenches": False}
    )
    return metadata["my_chip"]

In [ ]:
coverslip_dims = np.array([55e3, 24e3])
chip_dims0 = np.array([23e3, 13e3])
chip_dims1 = np.array([40e3, 20e3])
chip_dims2 = np.array([25e3, 16e3])

In [ ]:
plt.figure(figsize=(10, 5))
ax = plt.gca()
ax.add_patch(plt.Rectangle(-coverslip_dims / 2, *coverslip_dims, fill=False))
ax.add_patch(plt.Rectangle(-chip_dims0 / 2, *chip_dims0, fill=False, ls="-."))
ax.add_patch(plt.Rectangle(-chip_dims1 / 2, *chip_dims1, fill=False, ls="--"))
ax.add_patch(plt.Rectangle(-chip_dims2 / 2, *chip_dims2, fill=False))
ax.set_xlim(-1.2 * coverslip_dims[0] / 2, 1.2 * coverslip_dims[0] / 2)
ax.set_ylim(-1.2 * coverslip_dims[1] / 2, 1.2 * coverslip_dims[1] / 2)
ax.set_aspect("equal")

In [ ]:
fov = np.array([5056, 2960]) * 4.25 / 20

In [ ]:
metadata = {}
main_cell = s.chip(
    "Big Snake",
    s.snake,
    dims=chip_dims2,
    split=8,
    gap_lanes=1,
    feeding_channel_width=80,
    trench_spacing=2,
    trench_gap=20,
    trench_width=1.5,
    port_margin=1.5e3,
    top_margin=1.2e3,
    bottom_margin=1.2e3,
    horizontal_margin=4e3,
    draw_trenches=False,
    metadata=metadata,
)
fovs, unit_cell_height, active_height = fovs_per_chip(fov, **metadata["Big Snake"])
print(metadata)
print("FOVs: {} ({} x {})".format(np.product(fovs), fovs[0], fovs[1]))
print("unit cell height:", unit_cell_height)
print("active height: {} (margin: {})".format(active_height, fov[1] - active_height))
plot_cell(main_cell)

In [ ]:
# height: 1.4
# x2 S8 W1.3 L35 TS1
# x2 S8 W1.5 L35 TS1
# x1 S8 W1.5 L35 TS3
# x1 S8 W1.5 L45 TS2

# S8 W1.5 L35-65 sampler?

In [ ]:
%%time
metadata = {}
chip_dims = np.array([34e3, 18e3])
base_params = dict(
    design_func=s.snake,
    dims=chip_dims,
    split=8,
    gap_lanes=0,
    trench_length=35,
    # feeding_channel_width=40,
    horizontal_margin=2e3,
    trench_gap=20,
    trench_width=1.4,
    # trench_spacing=2.1,
    port_margin=0.6e3,
    top_margin=1.2e3,
    bottom_margin=0.6e3,
    draw_trenches=True,
    metadata=metadata,
)
manifold_params = dissoc(
    {
        **base_params,
        **dict(
            design_func=s.manifold_snake,
            dims=chip_dims,
            split=None,
            num_manifolds=1,
            manifold_width=200,
            manifold_input_margin=2e3,
            manifold_bend_margin=0.2e3,
            manifold_bend_radius=300,
            manifold_margin=100,
            feeding_channel_width=40,
            port_margin=0.5e3,
            top_margin=0.9e3,
            bottom_margin=0.9e3,
            # trench_spacing=2.1,
        ),
    },
    "horizontal_margin",
    "gap_lanes",
)
params = [
    dict(
        feeding_channel_width=80,
        trench_spacing=1.1,
        registration_marks=True,
        chip_id=0,
        mark_size=2,
        mark_spacing=1.5,
        tick_period=50,
        **base_params,
    ),
    dict(
        feeding_channel_width=80,
        trench_spacing=2.1,
        registration_marks=True,
        chip_id=0,
        mark_size=1,
        mark_spacing=1,
        tick_period=50,
        **base_params,
    ),
    dict(
        feeding_channel_width=80,
        trench_spacing=3.1,
        registration_marks=False,
        **base_params,
    ),
    dict(
        lanes_per_snake=3,
        trench_spacing=2.1,
        registration_marks=True,
        chip_id=1,
        mark_size=1,
        mark_spacing=1,
        **manifold_params,
    ),
    dict(
        lanes_per_snake=3,
        trench_spacing=2.1,
        registration_marks=False,
        **manifold_params,
    ),
    dict(
        lanes_per_snake=5,
        trench_spacing=2.1,
        registration_marks=True,
        chip_id=1,
        mark_size=1,
        mark_spacing=1,
        **manifold_params,
    )
    #     {
    #         **base_params,
    #         **dict(
    #             trench_width=1.5,
    #             trench_spacing=2.1,
    #             trench_length=45,
    #             trench_gap=15,
    #             feeding_channel_width=70,
    #             top_margin=1e3,
    #             bottom_margin=1e3,
    #         ),
    #     },
]
# params = [dict(split=4, fc=40, w=1.4, ts=1.6, tg=10), dict(split=4, fc=40, w=1.4, ts=1.1, tg=10), dict(split=4, fc=40, w=1.4, ts=1.1, tg=10),
#          dict(split=8, fc=40, w=1.4, ts=1.6, tg=10), dict(split=8, fc=40, w=1.4, ts=1.1, tg=10), dict(split=8, fc=40, w=1.4, ts=1.1, tg=10)]
chip_names = [
    "Basilisk {ports} FC{feeding_channel_width} L{trench_length} W{trench_width} TS{trench_spacing}{reg}".format(
        ports=f"S{p['split']}"
        if np.isscalar(p["split"])
        else "S{} LS{}".format(p["num_manifolds"], p["lanes_per_snake"]),
        reg=f" M{p['mark_size']}" if p["registration_marks"] else "",
        **p,
    )
    for p in params
]
chips = [s.chip(name, **p) for p, name in zip(params, chip_names)]
wafer_manifest = "\n".join([f"{idx+1}) " + name for idx, name in enumerate(chip_names)])
main_cell = s.wafer(
    chips,
    text_right="Basilisk \n JQS/DE/YG 200225.2",
    text_left=wafer_manifest,
    text=True,
    mask=False,
    chip_dims=chip_dims,
    diameter=(4 * u.inch).to(u.um).magnitude,
)
write_gds(main_cell, "200225.2basilisk.gds")

In [ ]:
# all: H75 L35 W1.4 TS2.1
# S8 FC80 x2
# S8 FC40 TS1
# S1 FC40 LS3 x2
# S1 FC40 LS5

In [ ]:
# print text summary for wafer spreadsheet

In [ ]:
metadata

In [ ]:
def get_resistance(feeding_channel_height, metadata):
    if "manifold_split" in metadata:
        res = manifold_flow_rates(feeding_channel_height, **metadata)
        R = np.array([r["R"] for r in res]) * np.asarray(metadata["manifold_split"])
    else:
        feeding_channel_height = 100  # * u.um
        split = metadata["split"]
        lane_length = metadata["lane_length"]
        feeding_channel_width = metadata["feeding_channel_width"]
        R = (
            np.asarray(split)
            * lane_length
            * resistance(feeding_channel_height, feeding_channel_width)
        )
    return R

In [ ]:
get_resistance(
    80, {"lane_length": 17920, "split": np.array([19]), "feeding_channel_width": 90}
)

In [ ]:
get_resistance(
    80, {"lane_length": 17920, "split": np.array([25]), "feeding_channel_width": 45}
)

In [ ]:
for name, md in metadata.items():
    R = get_resistance(75, md)
    print(f"{name}: {R}")

In [ ]:
print_fovs(fov_dims, metadata["Basilisk S1 LS5 FC40 L35 W1.4 TS2.1"])

In [ ]:
print_fovs(fov_dims, metadata["Basilisk S8 FC80 L35 W1.4 TS2.1"])